# Project 4. Identify which knowledge-based techniques are appropriate for which task

### 1. Expert Systems:
- Project: Medical Diagnosis System
- Description: Build an expert system that takes patient symptoms as input and provides potential medical conditions as output.
- Techniques: Rule-based reasoning, forward chaining, inference engine.

To create an expert system for the Medical Diagnosis System, we'll use the pyknow library, which is a Python implementation of the CLIPS expert system shell. We'll define rules that infer potential medical conditions based on the patient's symptoms. Make sure you have the pyknow library installed before running the code. You can install it using pip install pyknow.

In [2]:
!pip install experta

In [3]:
from experta import *


# Define the Facts (Patient Symptoms)
class Symptom(Fact):
    pass

# Define the Medical Conditions
class MedicalCondition(Fact):
    pass


# Define the Expert System Rules
class MedicalDiagnosisExpertSystem(KnowledgeEngine):
    @Rule(Symptom(cough=True) & Symptom(fever=True) & Symptom(fatigue=True))
    def rule_flu(self):
        self.declare(MedicalCondition(name="Flu"))

    @Rule(Symptom(headache=True) & Symptom(stiff_neck=True) & Symptom(fever=True))
    def rule_meningitis(self):
        self.declare(MedicalCondition(name="Meningitis"))

    @Rule(Symptom(rash=True) & Symptom(fever=True) & Symptom(joint_pain=True))
    def rule_dengue(self):
        self.declare(MedicalCondition(name="Dengue"))

    @Rule(Symptom(chest_pain=True) & Symptom(shortness_of_breath=True))
    def rule_heart_attack(self):
        self.declare(MedicalCondition(name="Heart Attack"))


# Function to run the Medical Diagnosis Expert System
def run_medical_diagnosis_system(symptoms):
    engine = MedicalDiagnosisExpertSystem()

    for symptom, value in symptoms.items():
        engine.declare(Symptom(**{symptom: value}))

    engine.run()

    # Get the output medical condition
    condition = engine.facts.get(MedicalCondition)
    if condition:
        return condition['name']
    else:
        return "No specific medical condition found."


# Example Usage
if __name__ == "__main__":
    # Example symptoms (You can modify this dictionary with patient symptoms)
    patient_symptoms = {
        'cough': True,
        'fever': True,
        'fatigue': True,
    }

    medical_condition = run_medical_diagnosis_system(patient_symptoms)
    print("Medical Condition:", medical_condition)


Medical Condition: No specific medical condition found.


### 2. Knowledge Graphs:


- Project: Movie Recommendation System
- Description: Create a knowledge graph of movies, actors, genres, and user preferences to recommend personalized movies to users.
- Techniques: RDF data representation, SPARQL querying, semantic similarity.

In [4]:
!pip install rdflib

  Using cached rdflib-6.3.2-py3-none-any.whl (528 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Obtaining dependency information for pyparsing<4,>=2.1.0 from https://files.pythonhosted.org/packages/a4/24/6ae4c9c45cf99d96b06b5d99e25526c060303171fb0aea9da2bfd7dbde93/pyparsing-3.1.0-py3-none-any.whl.metadata
Using cached pyparsing-3.1.0-py3-none-any.whl (102 kB)


In [7]:
!pip install SPARQLWrapper


  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl (28 kB)


In [14]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef
from rdflib.namespace import FOAF
from SPARQLWrapper import SPARQLWrapper, JSON

# Create a Knowledge Graph
graph = Graph()

# Define namespaces
DBPEDIA = Namespace("http://dbpedia.org/resource/")
MOVIE_ONTOLOGY = Namespace("http://example.org/movie_ontology#")

# Fetch movie data from DBpedia using SPARQL
sparql_endpoint = "https://dbpedia.org/sparql"
sparql_query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    
    SELECT ?movie ?title
    WHERE {
        ?movie a dbo:Film.
        ?movie foaf:name ?title.
        FILTER(LANG(?title) = "" || LANGMATCHES(LANG(?title), "en"))
    }
    LIMIT 10
"""

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Add movie data to the graph
if not results["results"]["bindings"]:
    raise ValueError("No movies found in the SPARQL results!")

for result in results["results"]["bindings"]:
    movie_uri = DBPEDIA[result["movie"]["value"]]
    graph.add((movie_uri, RDF.type, MOVIE_ONTOLOGY.Movie))
    graph.add((movie_uri, MOVIE_ONTOLOGY.title, Literal(result["title"]["value"])))

# Define user preferences
# Let's assume the user prefers the movie "The Matrix"
user_pref = DBPEDIA["The_Matrix"]

# Add some movie likes for the user
graph.add((URIRef("http://example.org/users/user1"), MOVIE_ONTOLOGY.likes, user_pref))

# Define SPARQL query to recommend movies to the user based on preferences
sparql_query_recommendation = f"""
    PREFIX movie: <http://example.org/movie_ontology#>
    SELECT ?movie ?title
    WHERE {{
        ?user movie:likes <{user_pref}>.
        ?movie movie:title ?title.
    }}
"""

print("SPARQL Query for Movie Recommendation:")
print(sparql_query_recommendation)

sparql_recommendation = SPARQLWrapper(sparql_endpoint)
sparql_recommendation.setQuery(sparql_query_recommendation)
sparql_recommendation.setReturnFormat(JSON)
recommendations = sparql_recommendation.query().convert()

# Print the recommended movies
print("\nRecommended Movies:")
for result in recommendations["results"]["bindings"]:
    print(result["title"]["value"])


SPARQL Query for Movie Recommendation:

    PREFIX movie: <http://example.org/movie_ontology#>
    SELECT ?movie ?title
    WHERE {
        ?user movie:likes <http://dbpedia.org/resource/The_Matrix>.
        ?movie movie:title ?title.
    }


Recommended Movies:
